In [1]:
import glob as gb
import cv2
import numpy as np

In [12]:
# img_path = gb.glob("D:\\BiometricsCoursework\\data\\training\\*.jpg")
img_path = gb.glob("training\\*.jpg")
side_view_face_cascade = cv2.CascadeClassifier("D:\\BiometricsCoursework\\opencv\\opencv\\sources\\data\\lbpcascades\\lbpcascade_profileface.xml")
frontal_face_cascade = cv2.CascadeClassifier("D:\\BiometricsCoursework\\opencv\\opencv\\sources\\data\\haarcascades\\haarcascade_frontalface_default.xml")

In [32]:
# Cut the head from picture
def CutHead(image):
    # Find the Highest Point
    m=0
    width=image.shape[1]
    heigh=image.shape[0]
    for i in range(0,heigh):
        for j in range(0,width):
            if image[i,j]==255:
                m=i
                break
        if m!=0:
            break
    img=image[m:heigh,:]
    width=img.shape[1]
    heigh=img.shape[0]
    # Find left point
    m=0
    for i in range(0,width):
        for j in range(0,heigh):
            if img[j,i]==255:
                m=i
                break
        if m!=0:
            break
    img=img[:,m:width]
    width=img.shape[1]
    heigh=img.shape[0]
    # Find right point
    m=0
    for i in range(width-1,-1,-1):
        for j in range(0,heigh):
            if img[j,i]==255:
                m=i
                break
        if m!=0:
            break
    # Cut head
    img=img[:,0:m]
    return img

In [36]:
for path in img_path:
    out_name = str(path)
    out_name = out_name[9:18]
    img = cv2.imread(path)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frontal_position = frontal_face_cascade.detectMultiScale(img2gray, 1.3, 5)
    if len(frontal_face_position) == 0:
        side_position = side_view_face_cascade.detectMultiScale(img2gray, 1.03, 5)

    # Transfer to YCrCb Colour space for better doing threshold
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Gray scale
    img = cv2.GaussianBlur(img, (9, 9), 0)  # Blur

    # Threshold
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    
    # If the input picture is frontal view of head
    if len(frontal_position) != 0:
        (x, y, w, h) = frontal_face_position[0]
        frontal_head = img[y - 70:y + h + 10, x - 40:x + w + 35]
        img=CutHead(frontal_head)
        cv2.imwrite(str(out_name) + ".png", img)

    # If the input picture is side view of head
    if len(frontal_position) == 0:
        (x, y, w, h) = side_position[0]
        side_view_head = img[y - 90:y + h, x - 40:x + w + 160]
        # Calculate features for side_view state
        img=CutHead(side_view_head)
        cv2.imwrite(str(out_name) + ".png", img)

In [8]:
# Cut full body from image
def cutbody(image):
    m=0
    heigh=image.shape[0]
    width=image.shape[1]
    for i in range(0,heigh):
        for j in range(0,width):
            if image[i,j]==255:
                m=i
                break
        if m!=0:
            break
    img=image[m:heigh,:]
    return img

In [17]:
from imageai.Detection import ObjectDetection
import os
# use imageAI
execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5"))
detector.loadModel()
i=0
for path in img_path:
    i=i+1
    execution_path = path
    custom_objects = detector.CustomObjects(person=True)
    detections = detector.detectCustomObjectsFromImage(custom_objects=custom_objects, input_image=os.path.join(path))
    # Get body position
    fullbody_position=detections[0]["box_points"]
    x=fullbody_position[0]
    y=fullbody_position[1]
    w=fullbody_position[2]
    h=fullbody_position[3]
    out_name = str(path)
    out_name = out_name[9:18]
    print(out_name)
    img = cv2.imread(path)
    img=img[y:y+h,x:x+w]
    # Transfer to YCrCb
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Gray scale
    img = cv2.GaussianBlur(img, (9, 9), 0)  # Blur
    # Threshold
    _, threshold = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    img=cutbody(threshold)
    cv2.imwrite(str(out_name) + ".png", img)


016z050pf
